In [1]:
import numpy as np
import pandas as pd
import pymannkendall as mkt
import timeit

# Timeit

https://docs.python.org/3/library/timeit.html#timeit.Timer

- time to run a statement (for a `number` times) is measured as **sec** unit


In [2]:
n = 100

a = np.arange(1, n+1)
mkt.original_test(a)

Mann_Kendall_Test(trend='increasing', h=True, p=0.0, z=14.73869998208331, Tau=1.0, s=4950.0, var_s=112750.0, slope=1.0, intercept=1.0)

In [3]:
setup_code = """
from __main__ import a
import pymannkendall as mkt
"""

test_code = """
mkt.original_test(a)
"""

num_exe = 1000

time = timeit.repeat(test_code, setup=setup_code, number=num_exe, repeat=3)
time_per_execution = np.mean(np.array(time)/num_exe)

print(f"{(time_per_execution*1000):.3f} ms, for {n} observations MKT")

1.050 ms, for 100 observations MKT


# Benchmarking MKT

- let's measure the time to run it

In [4]:
# generate observation sizes

magnitude = 10**np.arange(1, 6)
base = np.array([1, 2, 2.5, 4, 5, 7.5, 8])

num_obs = np.product(np.meshgrid(base, magnitude), axis=0)
num_obs = np.reshape(num_obs, np.product(num_obs.shape))
num_obs = np.array(num_obs, dtype=np.int)
num_obs

array([    10,     20,     25,     40,     50,     75,     80,    100,
          200,    250,    400,    500,    750,    800,   1000,   2000,
         2500,   4000,   5000,   7500,   8000,  10000,  20000,  25000,
        40000,  50000,  75000,  80000, 100000, 200000, 250000, 400000,
       500000, 750000, 800000])

In [5]:
num_obs = num_obs[num_obs <= 10000]

In [6]:
times = []

for size in num_obs:
    arr = np.arange(1, size+1, dtype=np.float)
    arr += np.random.rand(size)
    
    setup_code = "from __main__ import arr; import pymannkendall as mkt"

    test_code = "mkt.original_test(arr)"

    num_exe = 100
    num_repeat = 5

    time = timeit.repeat(test_code, setup=setup_code, number=num_exe, repeat=num_repeat)
    time_per_execution = np.mean(np.array(time)/num_exe)

    times.append(time_per_execution)
    
    print(f"{size:<10d} {(time_per_execution*1000):>10.3f} ms")

10              0.295 ms
20              0.370 ms
25              0.410 ms
40              0.544 ms
50              0.616 ms
75              0.846 ms
80              0.870 ms
100             1.059 ms
200             2.053 ms
250             2.707 ms
400             4.529 ms
500             6.020 ms
750            11.020 ms
800            12.633 ms
1000           15.024 ms
2000           49.005 ms
2500           67.650 ms
4000          190.672 ms
5000          243.779 ms
7500          503.761 ms
8000          568.948 ms
10000        1003.789 ms


In [9]:
df = pd.DataFrame({'n': num_obs, 'time_ms': np.round(np.array(times)*1000, 4)})

df

,n,time_ms
0,10,0.2947
1,20,0.3702
2,25,0.4105
3,40,0.5442
4,50,0.6162
5,75,0.8457
6,80,0.8702
7,100,1.0589
8,200,2.0528
9,250,2.7071


In [10]:
df.to_csv("./MKT_time_vs_n.csv", index=False)